# Tweets Analysis

In [1]:
import math
import numpy as np
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
import tweepy
import pickle
import json
def load_keys(path):
    """Loads your Twitter authentication keys from a file on disk.
    
    Args:
        path (str): The path to your key file.  The file should
          be in JSON format and look like this (but filled in):
            {
                "consumer_key": "<your Consumer Key here>",
                "consumer_secret":  "<your Consumer Secret here>",
                "access_token": "<your Access Token here>",
                "access_token_secret": "<your Access Token Secret here>"
            }
    
    Returns:
        dict: A dictionary mapping key names (like "consumer_key") to
          key values."""
    # Loading your keys from keys.json (which you should have filled
    # in in question 1):
    with open("keys.json") as f:
        keys = json.load(f)
        
    mapping = {}     
    mapping["consumer_key"] = keys["consumer_key"]
    mapping["consumer_secret"] = keys["consumer_secret"]
    mapping["access_token"] = keys["access_token"]
    mapping["access_token_secret"] = keys["access_token_secret"]
    return mapping

In [3]:
from twython import Twython # pip install twython
import time # standard lib

credentials = load_keys("keys.json")
twitter = Twython(credentials["consumer_key"],credentials["consumer_secret"],
                  credentials["access_token"],credentials["access_token_secret"])

In [4]:
def fetchData(user):
    user_timeline = twitter.get_user_timeline(screen_name=user,count=1)
    len(user_timeline)
    lis = [user_timeline[0]["id"]] ## this is the latest starting tweet id
    #i = 0
    with open("./prolife/{}.txt".format(user), "w", encoding='utf-8', errors='ignore') as f:
        try:
            for i in range(0, 16): ## iterate through all tweets
            ## tweet extract method with the last list item as the max_id
                user_timeline = twitter.get_user_timeline(screen_name=user,
                count=200, include_retweets=True, max_id=lis[-1])
                time.sleep(300) ## 5 minute rest between api calls
  
                for tweet in user_timeline:
                    #i += 1
                    f.write(tweet['text']) ## print the tweet
                    lis.append(tweet['id']) ## append tweet id's
        except:
            return

In [7]:
def download_recent_tweets_by_user(user_account_name, keys):
    """Downloads tweets by one Twitter user.

    Args:
        user_account_name (str): The name of the Twitter account
          whose tweets will be downloaded.
        keys (dict): A Python dictionary with Twitter authentication
          keys (strings), like this (but filled in):
            {
                "consumer_key": "<your Consumer Key here>",
                "consumer_secret":  "<your Consumer Secret here>",
                "access_token": "<your Access Token here>",
                "access_token_secret": "<your Access Token Secret here>"
            }

    Returns:
        list: A list of Status objects, each representing one tweet."""
    auth = tweepy.OAuthHandler(keys["consumer_key"], keys["consumer_secret"])
    auth.set_access_token(keys["access_token"], keys["access_token_secret"])
    api = tweepy.API(auth)

    # Getting as many recent tweets by user_account_name as Twitter will let us have:
    tweets_list = list(tweepy.Cursor(api.user_timeline, id=user_account_name).items())
    return tweets_list

def save_tweets_txt(tweets, path):
    """Saves a list of tweets to a file in the local filesystem.
    
    This function makes no guarantee about the format of the saved
    tweets, **except** that calling load_tweets(path) after
    save_tweets(tweets, path) will produce the same list of tweets
    and that only the file at the given path is used to store the
    tweets.  (That means you can implement this function however
    you want, as long as saving and loading works!)

    Args:
        tweets (list): A list of tweet objects (of type Status) to
          be saved.
        path (str): The place where the tweets will be saved.

    Returns:
        None"""
    # Saving the tweets to a file as "txt" objects:
    with open(path, "w", encoding='utf-8', errors='ignore') as f:
        for tweet in tweets:
            f.write(tweet.text) ## print the tweet

def save_tweets_pkl(tweets, path):
    """Saves a list of tweets to a file in the local filesystem.
    
    This function makes no guarantee about the format of the saved
    tweets, **except** that calling load_tweets(path) after
    save_tweets(tweets, path) will produce the same list of tweets
    and that only the file at the given path is used to store the
    tweets.  (That means you can implement this function however
    you want, as long as saving and loading works!)

    Args:
        tweets (list): A list of tweet objects (of type Status) to
          be saved.
        path (str): The place where the tweets will be saved.

    Returns:
        None"""
    # Saving the tweets to a file as "pickled" objects:
    with open(path, "wb") as outfile:
        pickle.dump(tweets, outfile)
        
def load_tweets(path):
    """Loads tweets that have previously been saved.
    
    Calling load_tweets(path) after save_tweets(tweets, path)
    will produce the same list of tweets.
    
    Args:
        path (str): The place where the tweets were be saved.

    Returns:
        list: A list of Status objects, each representing one tweet."""
    with open(path, "rb") as f:
        data = pickle.load(f)
    return data

def fetch_tweets(user_account_name, keys_path):
    """Get recent tweets from one user, loading from a disk cache if available.
    
    The first time you call this function, it will download tweets by
    a user.  Subsequent calls will not re-download the tweets; instead
    they'll load the tweets from a save file in your local filesystem.
    All this is done using the functions you defined in the previous cell.
    This has benefits and drawbacks that often appear when you cache data:
    
    +: Using this function will prevent extraneous usage of the Twitter API.
    +: You will get your data much faster after the first time it's called.
    -: If you really want to re-download the tweets (say, to get newer ones,
       or because you screwed up something in the previous cell and your
       tweets aren't what you wanted), you'll have to find the save file
       (which will look like <something>_recent_tweets.pkl) and delete it.
    
    Args:
        user_account_name (str): The Twitter handle of a user, without the @.
        keys_path (str): The path to a JSON keys file in your filesystem.
    """
    
    save_path_txt = "./prolife/{}.txt".format(user_account_name)
    save_path_pkl = "{}_recent_tweets.pkl".format(user_account_name)
    
    from pathlib import Path
    if not Path(save_path_pkl).is_file():
        keys = load_keys(keys_path)
        tweets = download_recent_tweets_by_user(user_account_name, keys)
        save_tweets_txt(tweets, save_path_txt)
        save_tweets_pkl(tweets, save_path_pkl)

In [10]:
users = [
"AbortionGroup",
"prolifecampaign",
"AcpAbortion",
"RosaryMovement",
"recallabortion",
"AbortionGroup",
"ExposeAbortion",
"40daysforlife",
"S2EAS_",
"LifeNewsHQ",
"AbolitionAHA",
"AntiAbrtionGang",
"FightAbortion",
"ProLifeBlogs",
"Bound4LIFE",
"HumanCoalition",
"LdnAntiAbortion",
"PLAM_org",
"ProLifeLSU",
"NeverAbortion",
"operationrescue",
"KeepLifeLegal",
"ProLifePolitics",
"ProLifeYouth",
"CA_ProLife"]

users = ["ProLifeYouth",
"CA_ProLife"]

In [11]:
for user in users:
    print("user:", user, "in process")
    time.sleep(400) ## 5 minute rest between api calls
    fetch_tweets(user, "keys.json")

user: ProLifeYouth in process
user: CA_ProLife in process


In [ ]:
for user in users:
    print("user:", user, "in process")
    fetchData(user)